## Machine Learning Analysis USA:
#### In this Jupyter Notebook you will find the process of building Machine Learning models and Evaluation of them on the file Clean_Insurance_USA.csv.

In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [174]:
usa = pd.read_csv('../Data/Clean_data/Clean_Insurance_USA.csv', index_col=0) #dataframe saved in us

In [175]:
usa.columns #Inspecting columns

Index(['Customer', 'State', 'Coverage', 'Education', 'Job_Status', 'Gender',
       'Income', 'Location', 'Civil_Status', 'Monthly_Price',
       'Months_LastClaim', 'Months_SinceActivation', 'Number_Open_Complaints',
       'Number_Insurances', 'Policy_Type', 'Sales_Channel', 'Car_Type',
       'Car_Size'],
      dtype='object')

Transformation of categorical data into boolean variables so that I can apply algorithms to it. 

In [176]:
dummy = pd.get_dummies(usa, columns = ['Civil_Status','Location', 'Policy_Type', 'Education', 'Job_Status',
                                       'Gender','Sales_Channel','Car_Type','Policy_Type','State','Coverage',
                                       'Car_Size'], drop_first = True)
usa_dummy = pd.concat([usa,dummy], axis=1)

Number_Open_Complaints is the feature that we want to predict, it is expressed in how many accidents did that person had in the last year, but for simplification, I will translate that into whether a customer had 1 or more accidents (1) or not (0).

In [177]:
usa['Number_Open_Complaints'] = usa.Number_Open_Complaints.apply(lambda x: 0 if x==0 else 1)

Separate our data in train and test, to check if our predictions are right or not.

In [178]:
from sklearn.model_selection import train_test_split #importing library

In [179]:
X_train, X_test, y_train, y_test = train_test_split(usa_dummy.drop(['Number_Open_Complaints','Customer','Income',
                                                                   'Monthly_Price','Months_LastClaim', 
                                                                    'Months_SinceActivation',
                                                                   'Civil_Status','Location', 
                                                                    'Policy_Type', 'Education', 
                                                                    'Job_Status','Gender','Sales_Channel',
                                                                    'Car_Type','Policy_Type','State','Coverage',
                                                                   'Car_Size'], axis = 1), 
                                                    usa[['Number_Open_Complaints']],
                                                    test_size=0.2, stratify = usa[['Number_Open_Complaints']])
#Used Stratify to make sure that proportion of 0 and 1s is uniform.

##### PCA

I have decided to let the model decide what are the common attributes, so I will be using PCA. I am loosing explanatory power but, this way, I am improving the accuracy power of my model.

In [180]:
from sklearn.decomposition import PCA 
  
pca = PCA(n_components = 3) 
  
X_train = pca.fit_transform(X_train) 
X_test = pca.transform(X_test) 
  
explained_variance = pca.explained_variance_ratio_ 

In [181]:
explained_variance #With these components, 80% of the variance is explained.

array([0.71163068, 0.03671994, 0.02411003])

#### Building Supervised Learning algorithms.
This is a case of supervised learning since we are trying to predict one outcome that is on the dataset and hence, I will build algorithms that maximizes the number of True Positives, people that might have an accident, always looking at the accuracy of the model.

##### Decision Tree Model

In [182]:
from sklearn import tree
from sklearn.metrics import confusion_matrix
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [183]:
prediction = clf.predict(X_test)

In [184]:
confusion_matrix(y_test, prediction) #All values are on the left (all Positives).

array([[1177,  274],
       [ 287,   89]])

In [185]:
clf.score(X_test, y_test)*100

69.29392446633827

In [195]:
from sklearn.metrics import f1_score, recall_score, precision_score
f1_score(y_test, prediction,average='weighted') #0.71 F1 score is good.

0.6909231377257328

In [196]:
precision_score(y_test, prediction)

0.24517906336088155

I have decided to dismiss this model since confusion matrix shows that this models does not predict well when there are accidents.

##### K-nearest Neighbours

In [188]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [189]:
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train.values.reshape(-1,))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [190]:
y_pred = knn.predict(X_test)

In [191]:
confusion_matrix(y_test, y_pred) #More distributed values, but low accuracy.

array([[1341,  110],
       [ 344,   32]])

In [192]:
knn.score(X_test, y_test)*100 #Good Score, 76.5% accuracy

75.15051997810619

In [219]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='weighted') #0.71 F1 score is good.

0.7100905024099323

In [220]:
recall_score(y_test, y_pred, average='weighted')

0.7783251231527094

In [221]:
precision_score(y_test,y_pred, average='weighted')

0.6881009473667226

Then, I decided to change the number of neighbours to see how accuracy and confusion matrix may change.

In [212]:
knn = KNeighborsClassifier(n_neighbors=6, metric='euclidean')
knn.fit(X_train, y_train.values.reshape(-1,))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

In [213]:
y_pred = knn.predict(X_test)

In [214]:
confusion_matrix(y_test, y_pred) #Worse confusion matrix.

array([[1405,   46],
       [ 359,   17]])

In [215]:
knn.score(X_test, y_test)*100 #Score of 79%

77.83251231527095

In [216]:
from sklearn.metrics import f1_score, recall_score, precision_score
f1_score(y_test, y_pred,average='weighted')

0.7100905024099323

In [217]:
recall_score(y_test, y_pred,average='weighted')

0.7783251231527094

In [218]:
precision_score(y_test, y_pred,average='weighted')

0.6881009473667226

I increased the number of neighbours to 7 and score decreased to 75%.

###### Conclusion
After analysing all the models below, I have checked all other models of supervised learning, but KNN seems to be the one that predicts better and has higher accuracy. So that's the model I will use.

##### Linear SVC

In [134]:
from sklearn.svm import LinearSVC

In [135]:
svc = LinearSVC()

In [136]:
svc.fit(X_train, y_train.values.reshape(-1,))

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [137]:
y_predi = svc.predict(X_test)
confusion_matrix(y_test, y_predi)

array([[1451,    0],
       [ 376,    0]])

I have decided to dismiss this model since confusion matrix shows that this models does not predict well when there are accidents.

##### SVC

In [138]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [139]:
confusion_matrix(y_test, y_pred) #It doesn't predict well in case there are accidents.

array([[1451,    0],
       [ 376,    0]])

I have decided to dismiss this model since confusion matrix shows that this models does not predict well when there are accidents.

##### Logistic Regression

In [140]:
from sklearn.linear_model import LogisticRegression

ks_model = LogisticRegression().fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [141]:
y_pred_test = ks_model.predict(X_test)
#y_pred_test
confusion_matrix(y_test, y_pred_test)

array([[1451,    0],
       [ 376,    0]])

I have decided to dismiss this model since confusion matrix shows that this models does not predict well when there are accidents.

In [142]:
from sklearn.linear_model import LogisticRegression   
  
classifier = LogisticRegression(random_state = 0) 
classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [143]:
y_pred = classifier.predict(X_test) 

In [144]:
confusion_matrix(y_test, y_pred)

array([[1451,    0],
       [ 376,    0]])

I have decided to dismiss this model since confusion matrix shows that this models does not predict well when there are accidents.

### CONCLUSION:
The model I have decided to choose was k-nearest neighbours with 5 neighbours since it is the best model in terms of accuracy and predictive power.